In [ ]:
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional

import tensorflow as tf
import keras.backend as K


## reload scripts before executing them
%load_ext autoreload
%autoreload 2

## importing python scripts from the directores

from dataset.reduce_memory import reduce_mem_usage
from dataset.create_windows import create_windows
from dataset.create_submission import create_submission_csv

from evaluation.rmsse_loss import root_mean_squared_scaled_error

from train.cross_validate import cross_validate

from models.lstm import lstm_model


import wandb
from wandb.keras import WandbCallback

In [ ]:
## Init Weights and Biases

wandb.init(sync_tensorboard=True, entity="whydoihavetoregister", project="m5_accuracy_forecasting")


In [ ]:
# read data

data_dir = 'data/'

train_sales = pd.read_csv(data_dir + 'sales_train_validation.csv')
#sell_prices = pd.read_csv(data_dir + 'sell_prices.csv')
calendar = pd.read_csv(data_dir + 'calendar.csv')
submission_file = pd.read_csv(data_dir + 'sample_submission.csv')
events_feature = pd.read_csv('eventencoding.csv')

In [ ]:
train_sales = reduce_mem_usage(train_sales) # takes about 4mins

In [ ]:
# create training data
sales = train_sales.drop(["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"], axis=1).T

# normalize
scaler = MinMaxScaler()
scaler.fit(sales)
sales = scaler.transform(sales)
sales = pd.DataFrame(sales)

In [ ]:
# create X and y

timesteps = 28
prediction_steps = 1

X,y = create_windows(sales, timesteps,prediction_steps)

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
## not sure about correctness of this loss, need to investigate more

rmse= root_mean_squared_scaled_error

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor = 0.1, patience = 2 , verbose = 1, mode= 'min', min_lr = 0.000001)


In [ ]:

n_features = X.shape[2]


In [ ]:
## create the model first

lstm_model = lstm_model(timesteps,n_features)


In [ ]:
cross_validate(lstm_model,X,y,batch_size=32,epochs = 2, number_of_folds = 2, callbacks=[reduce_lr,WandbCallback()])